rioxarray supports two geospatial metadata conventions for storing coordinate reference system (CRS) and transform information:

## CF Convention
xarray "... is particularly tailored to working with netCDF files, which were the source of xarray's data model..." (http://xarray.pydata.org).

For netCDF files, the GIS community uses CF conventions (http://cfconventions.org/). This stores geospatial metadata using grid_mapping coordinates.

Additionally, GDAL also supports these attributes:

- spatial_ref (Well Known Text)
- GeoTransform (GeoTransform array)

## Zarr Conventions
rioxarray now supports cloud-native conventions from the Zarr community:
- **Zarr spatial convention**: Stores transform and spatial metadata as direct attributes
- **Zarr proj convention**: Stores CRS information in multiple formats (code, WKT2, PROJJSON)

These conventions provide better performance for cloud storage and are more lightweight than CF conventions.

## Convention Selection
You can choose which convention to use:

```python
import rioxarray
from rioxarray import Convention

# Set global default (CF is default for backward compatibility)
rioxarray.set_options(convention=Convention.CF) 
rioxarray.set_options(convention=Convention.Zarr)

# Or specify per-method
data.rio.write_crs("EPSG:4326", convention=Convention.CF)
data.rio.write_crs("EPSG:4326", convention=Convention.Zarr)
```

References:

- CF: https://cfconventions.org/
- Zarr Spatial: https://github.com/zarr-conventions/spatial
- Zarr Proj: https://github.com/zarr-experimental/geo-proj
- Esri: https://pro.arcgis.com/en/pro-app/latest/help/data/multidimensional/spatial-reference-for-netcdf-data.htm
- GDAL: https://gdal.org/drivers/raster/netcdf.html#georeference
- pyproj: https://pyproj4.github.io/pyproj/stable/build_crs_cf.html

Operations on xarray objects can cause data loss. Due to this, rioxarray writes and expects the spatial reference information to exist appropriately based on the chosen convention.

## Accessing the CRS object

If you have opened a dataset and the Coordinate Reference System (CRS) can be determined, you can access it via the `rio.crs` accessor.

#### Search behavior:
The CRS reading follows the global convention setting from `rioxarray.set_options(convention=...)`:

**CF Convention (default)**:
1. Look in `encoding` of your data array for the `grid_mapping` coordinate name.
   Inside the `grid_mapping` coordinate first look for `spatial_ref` then `crs_wkt` and lastly the CF grid mapping attributes.
   This is in line with the Climate and Forecast (CF) conventions for storing the CRS as well as GDAL netCDF conventions.
2. Look in the `crs` attribute and load in the CRS from there. This is for backwards compatibility with `xarray.open_rasterio`, which is deprecated since version 0.20.0. We recommend using `rioxarray.open_rasterio` instead.

**Zarr Convention**:
1. Look for `proj:wkt2`, `proj:code`, or `proj:projjson` attributes on the data array (requires convention declaration in `zarr_conventions`)
2. For Datasets, check group-level `proj:*` attributes for inheritance

The value for the `crs` is anything accepted by `rasterio.crs.CRS.from_user_input()`

#### Search order for the CRS for Dataset:
If the CRS is not found using the search methods above, it also searches the `data_vars` and uses the
first valid CRS found.

#### decode_coords="all"

If you use one of xarray's open methods such as ``xarray.open_dataset`` to load netCDF files
with the default engine, it is recommended to use `decode_coords="all"`. This will load the grid mapping
variable into coordinates for compatibility with rioxarray.

#### API Documentation

- [rio.write_crs()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_crs)
- [rio.crs](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.crs)
- [rio.estimate_utm_crs()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.estimate_utm_crs)
- [rio.set_spatial_dims()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.set_spatial_dims)
- [rio.write_coordinate_system()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_coordinate_system)
- [rio.write_transform()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_transform)
- [rio.transform()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.transform)
- [rio.write_zarr_crs()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_zarr_crs) - New Zarr method
- [rio.write_zarr_transform()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_zarr_transform) - New Zarr method
- [rio.write_zarr_conventions()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_zarr_conventions) - New Zarr method

In [ ]:
import rioxarray  # activate the rio accessor
import xarray
from affine import Affine
from rioxarray import Convention

In [ ]:
rds = xarray.open_dataset("../../test/test_data/input/PLANET_SCOPE_3D.nc", decode_coords="all")

In [ ]:
rds.green.attrs

In [ ]:
rds.green.spatial_ref

In [ ]:
rds.green.rio.crs

## Setting the CRS

Use the `rio.write_crs` method to set the CRS on your `xarray.Dataset` or `xarray.DataArray`.
This modifies the `xarray.Dataset` or `xarray.DataArray` and sets the CRS based on the chosen convention.

### CF Convention
The CF convention stores metadata in grid_mapping coordinates, which is compatible with NetCDF and GDAL tools.

- [rio.write_crs()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_crs)
- [rio.crs](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.crs)

### Zarr Conventions
The Zarr conventions store metadata as direct attributes, providing better performance for cloud storage.

**Note:** It is recommended to use `rio.write_crs()` if you want the CRS to persist on the Dataset/DataArray and to write the CRS metadata. Calling only `rio.set_crs()` CRS storage method is lossy and will not modify the Dataset/DataArray metadata.

### Example: CF Convention (Default)

In [ ]:
xda = xarray.DataArray(1)
# CF convention is the default
xda.rio.write_crs(4326, inplace=True)
xda.spatial_ref

In [ ]:
xda.rio.crs

In [ ]:
# Show the grid_mapping attribute
print(f"grid_mapping: {xda.attrs.get('grid_mapping')}")
print(f"Has spatial_ref coordinate: {'spatial_ref' in xda.coords}")

### Example: Zarr Convention

In [ ]:
xda_zarr = xarray.DataArray(1)
# Use Zarr convention explicitly
xda_zarr.rio.write_crs(4326, convention=Convention.Zarr, inplace=True)

print(f"proj:code: {xda_zarr.attrs.get('proj:code')}")
print(f"zarr_conventions: {[c['name'] for c in xda_zarr.attrs.get('zarr_conventions', [])]}")
print(f"Has spatial_ref coordinate: {'spatial_ref' in xda_zarr.coords}")

In [ ]:
xda_zarr.rio.crs

### Example: Global Convention Setting

In [ ]:
# Set Zarr as global default
with rioxarray.set_options(convention=Convention.Zarr):
    xda_global = xarray.DataArray(1)
    xda_global.rio.write_crs(4326, inplace=True)  # Uses Zarr convention
    
    print(f"Global convention result:")
    print(f"proj:code: {xda_global.attrs.get('proj:code')}")
    print(f"Has grid_mapping: {'grid_mapping' in xda_global.attrs}")

## Spatial dimensions

Only 1-dimensional X and Y dimensions are supported.

The spatial dimension detection follows the global convention setting:

**Zarr Convention**:
- `spatial:dimensions` attribute (e.g., `["y", "x"]`)

**CF Convention**:
- x | y
- longitude | latitude  
- Coordinates (`coords`) with the CF attributes in `attrs`:
    - axis: X | Y
    - standard_name: longitude | latitude or projection_x_coordinate | projection_y_coordinate

Option 1: Write the CF attributes for non-standard dimension names

If you don't want to rename your dimensions/coordinates,
you can write the CF attributes so the coordinates can be found.

- [rio.set_spatial_dims()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.set_spatial_dims)
- [rio.write_coordinate_system()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_coordinate_system)

In [ ]:
rds.rio.write_crs(
    4326,
    inplace=True,
).rio.set_spatial_dims(
    x_dim="lon",
    y_dim="lat",
    inplace=True,
).rio.write_coordinate_system(inplace=True)

Option 2: Rename your coordinates

[xarray.Dataset.rename](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.rename.html)

In [ ]:
rds = rds.rename({"lon": "longitude", "lat": "latitude"}) 

## Setting the transform of the dataset

The transform can be calculated from the coordinates of your data.
This method is useful if your netCDF file does not have coordinates present.
Use the `rio.write_transform` method to set the transform on your `xarray.Dataset` or `xarray.DataArray`.

The transform storage follows the chosen convention:
- **CF Convention**: Stored as `GeoTransform` attribute on grid_mapping coordinate
- **Zarr Convention**: Stored as `spatial:transform` numeric array attribute

- [rio.write_transform()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_transform)
- [rio.transform()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.transform)
- [rio.write_zarr_transform()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_zarr_transform) - Zarr-specific method

### Example: CF Convention Transform

In [ ]:
transform = Affine(3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0)
xda.rio.write_transform(transform, convention=Convention.CF, inplace=True)
print(f"GeoTransform: {xda.spatial_ref.GeoTransform}")

In [ ]:
xda.rio.transform()

### Example: Zarr Convention Transform

In [ ]:
xda_zarr_transform = xarray.DataArray(1)
xda_zarr_transform.rio.write_transform(transform, convention=Convention.Zarr, inplace=True)
print(f"spatial:transform: {xda_zarr_transform.attrs.get('spatial:transform')}")
print(f"zarr_conventions: {[c['name'] for c in xda_zarr_transform.attrs.get('zarr_conventions', [])]}")

In [ ]:
xda_zarr_transform.rio.transform()

## Zarr-Specific Methods

rioxarray provides specialized methods for working with Zarr conventions:

In [ ]:
# Write CRS in multiple Zarr formats
sample_data = xarray.DataArray([[1, 2], [3, 4]], dims=["y", "x"])
zarr_all_formats = sample_data.rio.write_zarr_crs("EPSG:4326", format="all")

print(f"proj:code: {zarr_all_formats.attrs.get('proj:code')}")
print(f"Has proj:wkt2: {'proj:wkt2' in zarr_all_formats.attrs}")
print(f"Has proj:projjson: {'proj:projjson' in zarr_all_formats.attrs}")

In [ ]:
# Write complete Zarr conventions
complete_zarr = sample_data.rio.write_zarr_conventions(
    input_crs="EPSG:4326",
    transform=transform,
    crs_format="code"
)

print(f"Has CRS: {'proj:code' in complete_zarr.attrs}")
print(f"Has transform: {'spatial:transform' in complete_zarr.attrs}")
print(f"Has dimensions: {'spatial:dimensions' in complete_zarr.attrs}")
print(f"spatial:bbox: {complete_zarr.attrs.get('spatial:bbox')}")

## Summary

- **CF Convention (default)**: Traditional approach, widely compatible, uses grid_mapping coordinates
- **Zarr Conventions**: Zarr approach, better performance for cloud storage, uses direct attributes
- **Convention choice**: Set globally with `set_options()` or per-method with `convention=` parameter
- **Backward compatibility**: All existing code continues to work unchanged

For more examples and detailed information, see the [Geospatial Metadata Conventions](../conventions.rst) documentation.